# Tag 3 - Übung 1

In [34]:
import pandas as pd
import requests
import pprint as pp
import time

In [36]:
import cred 
GUARDIAN_KEY = cred.GUARDIAN_KEY

In [35]:
# API Endpoint
API_ENDPOINT = 'http://content.guardianapis.com/search' 

**Aufgabe 1:** Stellen Sie eine Anfrage an den Guardian API mit beliebigen Suchwörtern. Benutzen Sie dabei auch Suchoperatoren wie AND OR oder NOT. Begrenzen Sie den Suchzeitraum auf die letzten drei Monate. Informationen zu der Formulierung von 'search strings' finden Sie in der Dokumentation unter: https://open-platform.theguardian.com/documentation/

In [37]:
PARAMS = {
    'api-key': GUARDIAN_KEY,
    'from-date': '2023-05-01', 
    'to-date': '2023-07-07', 
    'lang': 'en', 
    'production-office': 'uk', 
    'q': 'russia AND war', 
    'show-fields': 'wordcount,body,byline' 
} 

In [38]:
# GET request
response = requests.get(API_ENDPOINT, params=PARAMS) 

# json Datei als dictionary speichern
response_dict = response.json()['response'] 

Wie sieht die URL der API Anfrage aus?

In [39]:
response.url

'http://content.guardianapis.com/search?api-key=ca445d55-8ebd-48f6-8a32-9d3a0a35e21b&from-date=2023-05-01&to-date=2023-07-07&lang=en&production-office=uk&q=russia+AND+war&show-fields=wordcount%2Cbody%2Cbyline'

Wie viele Artikel umfasst die Suchanfrage?

In [40]:
response_dict['total']

431

Auf wie vielen Seiten ist das Ergebnis aufgeteilt?

In [41]:
response_dict.keys()
response_dict['pages']

44

Extrahieren Sie nun alle Artikel Ihrer Suchanfrage. Vergrößern Sie dafür `page-size` auf 50.

In [42]:
PARAMS = {
    'api-key': GUARDIAN_KEY,
    'from-date': '2023-05-01', 
    'to-date': '2023-07-07', 
    'lang': 'en', 
    'production-office': 'uk', 
    'q': 'russia AND war', 
    'show-fields': 'wordcount,body,byline' ,
    'page-size': 50, # 50 Artikel je Seite
} 

response = requests.get(API_ENDPOINT, params=PARAMS) 
response_dict = response.json()['response']

Wie viele Seiten haben Sie jetzt?

In [43]:
response_dict['pages']

9

Extrahiern Sie alle Artikel mit einem while Loop. Orientieren Sie sich dabei an dem Script aus dem Manuscript

In [44]:
all_results = [] 
cur_page = 1 
total_pages = 1

while (cur_page <= total_pages) and (cur_page < 10): # with a fail safe 
    
    # Make a API request 
    PARAMS['page'] = cur_page 
    response = requests.get(API_ENDPOINT, params=PARAMS) 
    response_dict = response.json()['response'] 
    
    print(f"page: {cur_page} of {total_pages} - Articles (total): {response_dict['total']}")
    
    # Update our master results list 
    all_results += (response_dict['results']) 
    
    # Update our loop variables 
    total_pages = response_dict['pages'] 
    cur_page += 1 
    
    # sleep for 1 second
    time.sleep(1)
    
len(all_results)

page: 1 of 1 - Articles (total): 431
page: 2 of 9 - Articles (total): 431
page: 3 of 9 - Articles (total): 431
page: 4 of 9 - Articles (total): 431
page: 5 of 9 - Articles (total): 431
page: 6 of 9 - Articles (total): 431
page: 7 of 9 - Articles (total): 431
page: 8 of 9 - Articles (total): 431
page: 9 of 9 - Articles (total): 431


431

Speichern Sie die Ergebnisse in einem DataFrame

In [45]:
all_results_df = pd.json_normalize(all_results)
all_results_df.head()

,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,pillarName,fields.byline,fields.body,fields.wordcount
0,world/2023/jun/05/has-the-ukrainian-counteroff...,article,world,World news,2023-06-05T15:28:21Z,Has the Ukrainian counteroffensive begun in it...,https://www.theguardian.com/world/2023/jun/05/...,https://content.guardianapis.com/world/2023/ju...,False,pillar/news,News,Luke Harding,<p>For weeks now the Ukrainian army has been c...,1060
1,world/2023/jun/26/first-edition-putin-russia-w...,article,world,World news,2023-06-26T05:34:05Z,Monday briefing: How Russia pulled back from t...,https://www.theguardian.com/world/2023/jun/26/...,https://content.guardianapis.com/world/2023/ju...,False,pillar/news,News,Nimo Omer,<p>Good morning.</p> <p>Months of simmering te...,2320
2,world/2023/jun/07/chaos-on-frontlines-as-ukrai...,article,world,World news,2023-06-07T15:20:33Z,Chaos on frontlines as Ukraine war threatens t...,https://www.theguardian.com/world/2023/jun/07/...,https://content.guardianapis.com/world/2023/ju...,False,pillar/news,News,Andrew Roth and Pjotr Sauer,<p>If Vladimir Saldo was trying to project a s...,1178
3,commentisfree/2023/may/08/vladimir-putin-russi...,article,commentisfree,Opinion,2023-05-08T10:00:03Z,"As the Ukraine war grinds on, Russia is becomi...",https://www.theguardian.com/commentisfree/2023...,https://content.guardianapis.com/commentisfree...,False,pillar/opinion,Opinion,William Fear,<p>As much as Russia is the country of Tolstoy...,1227
4,world/2023/jun/25/wagner-chief-agrees-to-go-to...,article,world,World news,2023-06-25T15:22:25Z,Russia-Ukraine war at a glance: Putin appears ...,https://www.theguardian.com/world/2023/jun/25/...,https://content.guardianapis.com/world/2023/ju...,False,pillar/news,News,Charlie Moloney with Guardian staff and agencies,<ul> \n <li><p><strong>President Vladimir Puti...,709


Benennen Sie einige Spalten um

In [46]:
all_results_df = all_results_df.rename(columns={'webTitle':'article_title',
                                                'webUrl':'article_url',
                                                'fields.byline':'article_author',
                                               'webPublicationDate':'article_date'})
all_results_df.head()

,id,type,sectionId,sectionName,article_date,article_title,article_url,apiUrl,isHosted,pillarId,pillarName,article_author,fields.body,fields.wordcount
0,world/2023/jun/05/has-the-ukrainian-counteroff...,article,world,World news,2023-06-05T15:28:21Z,Has the Ukrainian counteroffensive begun in it...,https://www.theguardian.com/world/2023/jun/05/...,https://content.guardianapis.com/world/2023/ju...,False,pillar/news,News,Luke Harding,<p>For weeks now the Ukrainian army has been c...,1060
1,world/2023/jun/26/first-edition-putin-russia-w...,article,world,World news,2023-06-26T05:34:05Z,Monday briefing: How Russia pulled back from t...,https://www.theguardian.com/world/2023/jun/26/...,https://content.guardianapis.com/world/2023/ju...,False,pillar/news,News,Nimo Omer,<p>Good morning.</p> <p>Months of simmering te...,2320
2,world/2023/jun/07/chaos-on-frontlines-as-ukrai...,article,world,World news,2023-06-07T15:20:33Z,Chaos on frontlines as Ukraine war threatens t...,https://www.theguardian.com/world/2023/jun/07/...,https://content.guardianapis.com/world/2023/ju...,False,pillar/news,News,Andrew Roth and Pjotr Sauer,<p>If Vladimir Saldo was trying to project a s...,1178
3,commentisfree/2023/may/08/vladimir-putin-russi...,article,commentisfree,Opinion,2023-05-08T10:00:03Z,"As the Ukraine war grinds on, Russia is becomi...",https://www.theguardian.com/commentisfree/2023...,https://content.guardianapis.com/commentisfree...,False,pillar/opinion,Opinion,William Fear,<p>As much as Russia is the country of Tolstoy...,1227
4,world/2023/jun/25/wagner-chief-agrees-to-go-to...,article,world,World news,2023-06-25T15:22:25Z,Russia-Ukraine war at a glance: Putin appears ...,https://www.theguardian.com/world/2023/jun/25/...,https://content.guardianapis.com/world/2023/ju...,False,pillar/news,News,Charlie Moloney with Guardian staff and agencies,<ul> \n <li><p><strong>President Vladimir Puti...,709


Machen Sie den Text schön

In [47]:
from bs4 import BeautifulSoup

In [48]:
all_results_df['text'] = [BeautifulSoup(i, "html.parser").text for i in all_results_df['fields.body']]

In [49]:
all_results_df.head()

,id,type,sectionId,sectionName,article_date,article_title,article_url,apiUrl,isHosted,pillarId,pillarName,article_author,fields.body,fields.wordcount,text
0,world/2023/jun/05/has-the-ukrainian-counteroff...,article,world,World news,2023-06-05T15:28:21Z,Has the Ukrainian counteroffensive begun in it...,https://www.theguardian.com/world/2023/jun/05/...,https://content.guardianapis.com/world/2023/ju...,False,pillar/news,News,Luke Harding,<p>For weeks now the Ukrainian army has been c...,1060,For weeks now the Ukrainian army has been carr...
1,world/2023/jun/26/first-edition-putin-russia-w...,article,world,World news,2023-06-26T05:34:05Z,Monday briefing: How Russia pulled back from t...,https://www.theguardian.com/world/2023/jun/26/...,https://content.guardianapis.com/world/2023/ju...,False,pillar/news,News,Nimo Omer,<p>Good morning.</p> <p>Months of simmering te...,2320,Good morning. Months of simmering tension betw...
2,world/2023/jun/07/chaos-on-frontlines-as-ukrai...,article,world,World news,2023-06-07T15:20:33Z,Chaos on frontlines as Ukraine war threatens t...,https://www.theguardian.com/world/2023/jun/07/...,https://content.guardianapis.com/world/2023/ju...,False,pillar/news,News,Andrew Roth and Pjotr Sauer,<p>If Vladimir Saldo was trying to project a s...,1178,If Vladimir Saldo was trying to project a sens...
3,commentisfree/2023/may/08/vladimir-putin-russi...,article,commentisfree,Opinion,2023-05-08T10:00:03Z,"As the Ukraine war grinds on, Russia is becomi...",https://www.theguardian.com/commentisfree/2023...,https://content.guardianapis.com/commentisfree...,False,pillar/opinion,Opinion,William Fear,<p>As much as Russia is the country of Tolstoy...,1227,As much as Russia is the country of Tolstoy an...
4,world/2023/jun/25/wagner-chief-agrees-to-go-to...,article,world,World news,2023-06-25T15:22:25Z,Russia-Ukraine war at a glance: Putin appears ...,https://www.theguardian.com/world/2023/jun/25/...,https://content.guardianapis.com/world/2023/ju...,False,pillar/news,News,Charlie Moloney with Guardian staff and agencies,<ul> \n <li><p><strong>President Vladimir Puti...,709,\nPresident Vladimir Putin appeared on Russian...


Filtern Sie die 'wichtigsten' Spalten und speichern Sie die Tabelle als csv Datei

In [50]:
all_results_df_f = all_results_df[['article_title', 'article_date', 'article_author', 'text']].copy()

In [51]:
all_results_df_f.to_csv('../data/guardian_texts_russia.csv', index=False)